In [16]:
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
import os
import time

In [17]:
#Getting Data
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'Length of text:{len(text)} characters')

Length of text:1115394 characters


In [18]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [19]:
#Text pre processing
#Need to convert text to a numerical representation
#Makes each character into numeric ID

ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None) #defined a layer to do conversion
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None) #To get letters from ids


#Define a function to join the characters to make a string
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [20]:
#Data labelling
#the input is contain seq_length # of characters, and so does the target
#therefore break into chunks of seq_length+1

#convert text vector into a stream of character indices
all_ids = ids_from_chars(tf.strings.unicode_split(text,'UTF-8'))
seq_length=100
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

#mapping to (input,label) pairs
def split_input_target(sequence):
    input_text=sequence[:-1]
    target_text=sequence[1:]
    return input_text,target_text

dataset=sequences.map(split_input_target)

In [21]:
#BATCHING

BATCH_SIZE = 64

BUFFER_SIZE= 1000 #buffer in which it shuffles element
dataset=(
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)

)
dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [24]:
#MODEL

vocab_size=len(ids_from_chars.get_vocabulary())
embedding_dim=256
rnn_units=1024

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [25]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [26]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [28]:
#Training
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/20
172/172 [==============================] - 5s 22ms/step - loss: 2.7057
Epoch 2/20
172/172 [==============================] - 4s 21ms/step - loss: 1.9952
Epoch 3/20
172/172 [==============================] - 4s 22ms/step - loss: 1.7271
Epoch 4/20
172/172 [==============================] - 4s 21ms/step - loss: 1.5607
Epoch 5/20
172/172 [==============================] - 4s 21ms/step - loss: 1.4551
Epoch 6/20
172/172 [==============================] - 4s 22ms/step - loss: 1.3840
Epoch 7/20
172/172 [==============================] - 4s 22ms/step - loss: 1.3289
Epoch 8/20
172/172 [==============================] - 4s 21ms/step - loss: 1.2827
Epoch 9/20
172/172 [==============================] - 4s 22ms/step - loss: 1.2393
Epoch 10/20
172/172 [==============================] - 4s 22ms/step - loss: 1.1978
Epoch 11/20
172/172 [==============================] - 4s 22ms/step - loss: 1.1563
Epoch 12/20
172/172 [==============================] - 4s 22ms/step - loss: 1.1128
Epoch 13/20
1

In [30]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Three in another seful.
Hath not been in me; and she knave is his.

GLOUCESTER:
What ever I acquice the meat, Pomfort; no harm?

GREMIO:
I am done, or else will I be your own.

ALONSO:
But they are soldier, as they do,
I would they desire them upon an infringed womp;
'Tis but here pounday, and a bale in fantasticat,
O God, thou shalt let her beseeming her,
She for such venwees to the appeal, back'd and
one, I would hang him. But, out on our house,
By heather for your excellant potrock:
Holder, farewell; one whoreson, thou shouldst
be patripious at the irland. On he did but ever
so hardly as heaven, by any of this once.

LEONTES:
Thou art to hear.
Can I beseech you, Kate: none care you off.
Here in Vienna, sir, but us in all: but hath
cerved'd me his hands
Out of this good and sit in all;
Of where he stands too burn a hoal!
Moved. What elce, I have overheeld you.

VOLUMNIA:
I would do thee this roar file week:
This fellow I the dear'st o' both of weather.
Courage all agree, do In